In [ ]:
!pip install tflearn
!pip install tensorflow

In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tflearn
import tensorflow
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
with open("intents_test_pl.json") as file:
  data = json.load(file)

  print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Cześć', 'Elo', 'Jesteś?', 'Hej', 'Dzień dobry', 'Sup'], 'responses': ['Cześć!', 'Dobrze Cię widzieć!', 'Hej, w czym mogę pomóc?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['narazie', 'Do zobaczenia', 'Dowidzenia', 'Dobranoc', 'Miłego dnia'], 'responses': ['Do zobaczenia później', 'Mam nadzieję, że później pogadamy', 'Narazie!'], 'context_set': ''}, {'tag': 'age', 'patterns': ['Ile masz lat', 'Ile lat ma Janet', 'Wiek', 'Jak stara jesteś', 'urodziny'], 'responses': ['Mam kilka dni', 'Urodziłam się 17.03.2021'], 'context_set': ''}, {'tag': 'name', 'patterns': ['Jak masz na imię', 'Jak Cię zwą?', 'twoje imie?', 'Imie', 'Jak cię nazywać', 'Kim jesteś'], 'responses': ['Możesz mnie nazywać Janet!', 'Jestem Janet', 'Jestem Janet, twój ulubiony chatbot'], 'context_set': ''}]}


In [ ]:
words = []
labels = []
docs_x = []
docs_y = []

In [ ]:
for intent in data["intents"]: #loop through all the dictionaries
  for pattern in intent["patterns"]: #loop through all the patterns in each intent
    wrds = nltk.word_tokenize(pattern) #Tokenizing every word
    words.extend(wrds) #Add every tokenized word
    docs_x.append(wrds) #Add the pattern name
    docs_y.append(intent["tag"]) #Pattern x coresponds to the tag y

  if intent["tag"] not in labels:
    labels.append(intent["tag"]) #Add the tag

BONUS Polish Stemmer

In [ ]:
!pip install pystempel

In [ ]:
from stempel import StempelStemmer

stemmer_pl = StempelStemmer.default() #może wersja ".polimorf()" jest lepsza?

Loading: 100%|██████████| 2225192/2225192 [00:02<00:00, 746788.60bytes/s]


In [ ]:
print(stemmer_pl.stem("książką"))

książka


END

In [ ]:
words = [stemmer_pl.stem(w.lower()) for w in words if w not in "?"] #stemming -> take each word and bring it to the "root" form. Only the main meaning of the word is important to us
words = sorted(list(set(words))) #Sorting

labels = sorted(labels) #sorting

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

In [ ]:
for x, doc in enumerate(docs_x):
  bag =[]

  wrds = [stemmer_pl.stem(w) for w in doc]

  for w in words:
    if w in wrds:
      bag.append(1) #this word exist
    else:
      bag.append(0)
    
  output_row = out_empty[:]
  output_row[labels.index(docs_y[x])] = 1

  training.append(bag)
  output.append(output_row)

training = numpy.array(training)
output = np.array(output)

### Model

In [ ]:
training = numpy.array(training) #list into array to feed the model
output = numpy.array(output) #list into array to feed the model

In [ ]:
tensorflow.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])]) #Input layer
net = tflearn.fully_connected(net, 8) #16 neurons for hidden layer
net = tflearn.fully_connected(net, 8) #16 neurons for hidden layer
net = tflearn.fully_connected(net, 8) #16 neurons for hidden layer
net = tflearn.fully_connected(net, len(output[0]), activation="softmax") #len(output) neurons for output layer
net = tflearn.regression(net)

model = tflearn.DNN(net)

model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 2999  | total loss: 0.54356 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.54356 - acc: 0.7546 -- iter: 16/22
Training Step: 3000  | total loss: 0.53645 | time: 0.015s
| Adam | epoch: 1000 | loss: 0.53645 - acc: 0.7458 -- iter: 22/22
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


### User input

In [ ]:
def bag_of_words(s, words):
  bag = [0 for _ in range(len(words))]

  s_words = nltk.word_tokenize(s)
  s_words = [stemmer_pl.stem(word.lower()) for word in s_words]

  for se in s_words:
    for i, w in enumerate(words):
      if w == se:
        bag[i] = 1
  return np.array(bag)

In [ ]:
def chat():
  print("Start talking with the bot! (po polacku) (type quit to stop)")
  while True:
    inp = input("You: ")
    if inp.lower() == "quit":
      break

    result = model.predict([bag_of_words(inp,words)])
    result_index = numpy.argmax(result)
    tag = labels[result_index]
    
    for tg in data["intents"]:
      if tg['tag'] == tag:
        responses = tg['responses']
      
    print(random.choice(responses))


In [ ]:
chat()

Start talking with the bot! (po polacku) (type quit to stop)
You: hej
Hej, w czym mogę pomóc?
You: ile masz lat?
Mam kilka dni
You: narazie
Do zobaczenia później
You: co robisz?
Cześć!
You: jak masz na imię?
Jestem Janet, twój ulubiony chatbot
You: quit
